# TFM: Analisis de Topicos en 10-k - Empresas del SP500

## Parte 1: Descarga de 10-k de las compañias

In [5]:
import os
import re
import shutil
from pathlib import Path
from sec_edgar_downloader import Downloader
from typing import List, Dict
from datetime import datetime
import pandas as pd
from bs4 import BeautifulSoup
import sys

# AÑADIDO PARA UTF-8 EN CONSOLA WINDOWS
#sys.stdout.reconfigure(encoding='utf-8')

EMAIL = "tu_email@ejemplo.com"  # Correo para SEC EDGAR
FORM_TYPE = "10-K"  # Tipo de formulario
START_YEAR = 2016  # Año inicial
CURRENT_YEAR = 2025  # Año final

BASE_DIR = "D:/TFM/Analisis_10k"  # o "D:\\10K_Data" si prefieres doble barra

INPUT_ROOT = f"{BASE_DIR}/Input"
TEMP_DIR = f"{BASE_DIR}/temp_10k_filings"
COMPANIES_EXCEL = f"{BASE_DIR}/companies_cik.xlsx"

# Empresas faltantes
#COMPANIES_EXCEL = f"{BASE_DIR}/companies_cik_faltante.xlsx"

def load_companies_from_excel(excel_path: str) -> List[Dict]:
    """Carga la información de las empresas desde un archivo Excel."""
    try:
        # Leer el Excel, asegurando que 'cik' sea string
        df = pd.read_excel(excel_path, dtype={'cik': str})
        
        # Convertir el DataFrame a una lista de diccionarios
        companies = df.to_dict('records')
        
        # Verificar que las columnas requeridas estén presentes
        required_columns = ['company_name', 'cik', 'company_dir']
        if not all(col in df.columns for col in required_columns):
            missing = [col for col in required_columns if col not in df.columns]
            raise ValueError(f"Faltan columnas en el Excel: {missing}")
        
        return companies
    except Exception as e:
        print(f"Error al leer el archivo Excel {excel_path}: {e}")
        return []

def extract_filing_date(file_path: Path) -> str:
    """Extrae la fecha de presentación del archivo 10-K, soportando texto y HTML."""
    try:
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
            text = f.read()
        match = re.search(r'<FILING-DATE>(\d{8})', text)
        if match:
            return pd.to_datetime(match.group(1), format='%Y%m%d').strftime('%Y-%m-%d')
        match = re.search(r'<ACCEPTANCE-DATETIME>(\d{8})', text)
        if match:
            return pd.to_datetime(match.group(1), format='%Y%m%d').strftime('%Y-%m-%d')
        soup = BeautifulSoup(text, 'html.parser')
        for tag in soup.find_all(text=re.compile(r'(FILING-DATE|ACCEPTANCE-DATETIME)')):
            match = re.search(r'\d{8}', tag)
            if match:
                return pd.to_datetime(match.group(0), format='%Y%m%d').strftime('%Y-%m-%d')
        print(f"No se encontró fecha en {file_path}")
        return None
    except Exception as e:
        print(f"Error extrayendo fecha de {file_path}: {e}")
        return None

def move_to_date_folder(file_path: Path, company_dir: str, filing_date: str) -> Path:
    """Mueve el archivo a la carpeta Input/company/date y retorna la nueva ruta."""
    target_dir = Path(INPUT_ROOT) / company_dir / filing_date
    target_dir.mkdir(parents=True, exist_ok=True)
    target_path = target_dir / file_path.name
    file_path.rename(target_path)
    return target_path

def clean_temp_dir(temp_dir: Path):
    """Elimina todos los archivos y carpetas en el directorio temporal."""
    if temp_dir.exists():
        shutil.rmtree(temp_dir, ignore_errors=True)
        print(f"Eliminado directorio temporal: {temp_dir}")
    temp_dir.mkdir(parents=True, exist_ok=True)

def download_10k_filings(
    companies: List[Dict],
    email: str,
    form_type: str,
    start_year: int,
    current_year: int,
    input_root: str,
    temp_dir: str
) -> None:
    """Descarga formularios 10-K y los organiza en Input/company/date."""
    temp_dir_path = Path(temp_dir)
    
    # Limpiar el directorio temporal al inicio
    clean_temp_dir(temp_dir_path)
    
    try:
        for company in companies:
            company_name = company["company_name"]
            cik = company["cik"]
            company_dir = company["company_dir"]
            dl = Downloader(company_name, email, temp_dir)
            
            # Descargar archivos año por año
            for year in range(start_year, current_year + 1):
                try:
                    print(f"Descargando {form_type} para {company_name}, año {year}...")
                    dl.get(form_type, cik, after=f"{year}-01-01", before=f"{year}-12-31")
                    print(f"Descarga completada para {company_name}, año {year}.")
                except Exception as e:
                    print(f"Error descargando {form_type} para {company_name}, año {year}: {e}")
            
            # Procesar archivos descargados
            temp_company_dir = Path(temp_dir) / "sec-edgar-filings" / str(cik) / form_type
            if not temp_company_dir.exists():
                print(f"No se encontraron archivos para {company_name} en {temp_company_dir}")
                continue
            
            for file_path in temp_company_dir.rglob("*.*"):
                filing_date = extract_filing_date(file_path)
                if not filing_date:
                    print(f"Omitiendo {file_path}: No se encontró fecha de presentación")
                    continue
                
                # Mover archivo y eliminarlo del directorio temporal
                new_path = move_to_date_folder(file_path, company_dir, filing_date)
                print(f"Movido {file_path} -> {new_path}")
                try:
                    file_path.unlink(missing_ok=True)  # Eliminar archivo temporal
                except Exception as e:
                    print(f"Error eliminando {file_path}: {e}")
            
            # Limpiar directorio temporal de la compañía
            try:
                shutil.rmtree(temp_company_dir, ignore_errors=True)
                print(f"Eliminado directorio temporal: {temp_company_dir}")
            except Exception as e:
                print(f"Error eliminando {temp_company_dir}: {e}")
    
    finally:
        # Limpiar TEMP_DIR al finalizar (en caso de error)
        clean_temp_dir(temp_dir_path)

def main():
    """Función principal para descargar y organizar formularios 10-K."""
    # Cargar empresas desde el Excel
    companies = load_companies_from_excel(COMPANIES_EXCEL)
    if not companies:
        print("No se cargaron empresas. Terminando ejecución.")
        return
    download_10k_filings(companies, EMAIL, FORM_TYPE, START_YEAR, CURRENT_YEAR, INPUT_ROOT, TEMP_DIR)

if __name__ == "__main__":
    main()

Eliminado directorio temporal: D:\TFM\Analisis_10k\temp_10k_filings
Descargando 10-K para Brown_Forman, año 2016...
Descarga completada para Brown_Forman, año 2016.
Descargando 10-K para Brown_Forman, año 2017...
Descarga completada para Brown_Forman, año 2017.
Descargando 10-K para Brown_Forman, año 2018...
Descarga completada para Brown_Forman, año 2018.
Descargando 10-K para Brown_Forman, año 2019...
Descarga completada para Brown_Forman, año 2019.
Descargando 10-K para Brown_Forman, año 2020...
Descarga completada para Brown_Forman, año 2020.
Descargando 10-K para Brown_Forman, año 2021...
Descarga completada para Brown_Forman, año 2021.
Descargando 10-K para Brown_Forman, año 2022...
Descarga completada para Brown_Forman, año 2022.
Descargando 10-K para Brown_Forman, año 2023...
Descarga completada para Brown_Forman, año 2023.
Descargando 10-K para Brown_Forman, año 2024...
Descarga completada para Brown_Forman, año 2024.
Descargando 10-K para Brown_Forman, año 2025...
Descarga co

## Parte 2: Crear archivo con metadatos donde se almacenenen las rutas de los archivos descargados

In [10]:
## Parte 2: Crear archivo con metadatos donde se almacenenen las rutas de los archivos descargados

In [8]:
import re
import pandas as pd
from pathlib import Path
from typing import List, Dict
from datetime import datetime
from bs4 import BeautifulSoup

BASE_DIR = "D:/TFM/Analisis_10k"  # o "D:\\10K_Data" si prefieres doble barra

OUTPUT_EXCEL = f"{BASE_DIR}/10k_filings_metadata.xlsx"
COMPANIES_EXCEL = f"{BASE_DIR}/companies_cik.xlsx"



def load_companies_from_excel(excel_path: Path) -> List[Dict]:
    try:
        df = pd.read_excel(excel_path)
        companies = df.to_dict('records')

        required_columns = ['company_name', 'cik', 'input_dir', 'output_dir']
        if not all(col in df.columns for col in required_columns):
            missing = [col for col in required_columns if col not in df.columns]
            raise ValueError(f"Faltan columnas en el Excel: {missing}")
        
        return companies
    except Exception as e:
        print(f"Error al leer el archivo Excel {excel_path}: {e}")
        return []

def extract_10k_text(text: str) -> str:
    documents = re.findall(r'<DOCUMENT>(.*?)</DOCUMENT>', text, re.DOTALL)
    for doc in documents:
        type_match = re.search(r'<TYPE>(.+)', doc)
        if type_match and type_match.group(1).strip().upper() == '10-K':
            text_match = re.search(r'<TEXT>(.*?)</TEXT>', doc, re.DOTALL)
            if text_match:
                return text_match.group(1)

    soup = BeautifulSoup(text, 'html.parser')
    documents = soup.find_all('document')
    for doc in documents:
        doc_type = doc.find('type')
        if doc_type and doc_type.text.strip().upper() == '10-K':
            text_tag = doc.find('text')
            if text_tag:
                return text_tag.text
    return None

def extract_filing_date(original_text: str) -> datetime:
    match = re.search(r'<FILING-DATE>(\d{8})', original_text)
    if match:
        return pd.to_datetime(match.group(1), format='%Y%m%d')
    match = re.search(r'<ACCEPTANCE-DATETIME>(\d{8})', original_text)
    if match:
        return pd.to_datetime(match.group(1), format='%Y%m%d')

    soup = BeautifulSoup(original_text, 'html.parser')
    for tag in soup.find_all(text=re.compile(r'(FILING-DATE|ACCEPTANCE-DATETIME)')):
        match = re.search(r'\d{8}', tag)
        if match:
            return pd.to_datetime(match.group(0), format='%Y%m%d')
    return None

def clean_and_save_filings(
    input_dir: str,
    output_dir: str,
    company_name: str,
    cik: str
) -> List[Dict]:
    
    filings = []
    input_path = Path(input_dir)
    output_base = Path(output_dir)

    for file_path in input_path.rglob("*.txt"):
        filing_date_str = file_path.parent.name
        try:
            filing_date = datetime.strptime(filing_date_str, "%Y-%m-%d")
        except ValueError:
            print(f"Omitiendo {file_path}: Nombre de carpeta no es una fecha válida")
            continue

        try:
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                original_text = f.read()

            clean_text = extract_10k_text(original_text)
            if not clean_text:
                print(f"Omitiendo {file_path}: No se extrajo texto 10-K válido")
                continue

            output_dir = output_base / filing_date_str
            output_dir.mkdir(parents=True, exist_ok=True)

            filing_year = filing_date.year
            output_filename = f"{company_name.replace(' ', '_')}_{filing_year}_clean.txt"
            output_file_path = output_dir / output_filename

            with open(output_file_path, 'w', encoding='utf-8') as f_out:
                f_out.write(clean_text)

            filings.append({
                "date": filing_date,
                "year": filing_year,
                "company": company_name,
                "CIK": cik,
                "file_path": str(output_file_path)
            })
            print(f"Procesado {file_path} -> {output_file_path}")

        except Exception as e:
            print(f"Error procesando {file_path}: {e}")

    return filings


def main():
    companies = load_companies_from_excel(COMPANIES_EXCEL)
    if not companies:
        print("No se cargaron empresas. Terminando ejecución.")
        return

    all_cleaned_filings = []
    for company in companies:
        # Convertimos rutas relativas a absolutas con respecto a BASE_DIR
        input_dir = str(Path(BASE_DIR) / company["input_dir"].lstrip("/\\"))
        output_dir = str(Path(BASE_DIR) / company["output_dir"].lstrip("/\\"))

        cleaned = clean_and_save_filings(
            input_dir=input_dir,
            output_dir=output_dir,
            company_name=company["company_name"],
            cik=company["cik"]
        )
        all_cleaned_filings.extend(cleaned)

    if all_cleaned_filings:
        df_10k_clean = pd.DataFrame(all_cleaned_filings)
        try:
            df_10k_clean.to_excel(OUTPUT_EXCEL, index=False)
            print(f"Metadatos guardados en {OUTPUT_EXCEL}")
        except Exception as e:
            print(f"Error guardando metadatos en {OUTPUT_EXCEL}: {e}")
        return df_10k_clean
    else:
        print("No se procesaron formularios. No se creó el archivo Excel.")
        return None

if __name__ == "__main__":

    
    df_10k_clean = main()


Procesado D:\TFM\Analisis_10k\Input\BlackRock\2025-02-25\full-submission.txt -> D:\TFM\Analisis_10k\Input_clean\BlackRock\2025-02-25\BlackRock_2025_clean.txt
Procesado D:\TFM\Analisis_10k\Input\Smurfit_Westrock\2025-03-07\full-submission.txt -> D:\TFM\Analisis_10k\Input_clean\Smurfit_Westrock\2025-03-07\Smurfit_Westrock_2025_clean.txt
Procesado D:\TFM\Analisis_10k\Input\Bunge_Global\2024-02-22\full-submission.txt -> D:\TFM\Analisis_10k\Input_clean\Bunge_Global\2024-02-22\Bunge_Global_2024_clean.txt
Procesado D:\TFM\Analisis_10k\Input\Bunge_Global\2025-02-20\full-submission.txt -> D:\TFM\Analisis_10k\Input_clean\Bunge_Global\2025-02-20\Bunge_Global_2025_clean.txt
Procesado D:\TFM\Analisis_10k\Input\GE_Vernova\2025-02-06\full-submission.txt -> D:\TFM\Analisis_10k\Input_clean\GE_Vernova\2025-02-06\GE_Vernova_2025_clean.txt
Procesado D:\TFM\Analisis_10k\Input\TKO_Group_Holdings\2024-02-27\full-submission.txt -> D:\TFM\Analisis_10k\Input_clean\TKO_Group_Holdings\2024-02-27\TKO_Group_Holding

In [13]:
df_10k_clean.shape

(4717, 5)

In [17]:
import re
import pandas as pd
from pathlib import Path
from typing import List, Dict
from datetime import datetime
from bs4 import BeautifulSoup

BASE_DIR = "D:/TFM/Analisis_10k"

OUTPUT_EXCEL = f"{BASE_DIR}/10k_filings_metadata2.xlsx"
COMPANIES_EXCEL = f"{BASE_DIR}/companies_cik.xlsx"

def load_companies_from_excel(excel_path: Path) -> List[Dict]:
    try:
        df = pd.read_excel(excel_path)
        companies = df.to_dict('records')

        required_columns = ['company_name', 'cik', 'input_dir', 'output_dir']
        if not all(col in df.columns for col in required_columns):
            missing = [col for col in required_columns if col not in df.columns]
            raise ValueError(f"Faltan columnas en el Excel: {missing}")
        
        return companies
    except Exception as e:
        print(f"Error al leer el archivo Excel {excel_path}: {e}")
        return []

def extract_10k_text(text: str) -> str:
    documents = re.findall(r'<DOCUMENT>(.*?)</DOCUMENT>', text, re.DOTALL)
    for doc in documents:
        type_match = re.search(r'<TYPE>(.+)', doc)
        if type_match and type_match.group(1).strip().upper() == '10-K':
            text_match = re.search(r'<TEXT>(.*?)</TEXT>', doc, re.DOTALL)
            if text_match:
                return text_match.group(1)

    soup = BeautifulSoup(text, 'html.parser')
    documents = soup.find_all('document')
    for doc in documents:
        doc_type = doc.find('type')
        if doc_type and doc_type.text.strip().upper() == '10-K':
            text_tag = doc.find('text')
            if text_tag:
                return text_tag.text
    return None

def clean_html_with_bs(text: str) -> str:
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text(separator=' ', strip=True)

def extract_relevant_text(text: str, phrase: str = "financial condition") -> str:
    if not text or not phrase:
        return text
    index = text.lower().find(phrase.lower())
    return text[index:].strip() if index != -1 else text.strip()

#def extract_item_7_to_7a(text: str) -> str:
#    pattern = r'(?s)(ITEM\s*7\.)(.*?)(?=ITEM\s*7a\.)'
#    match = re.search(pattern, text, re.IGNORECASE)
#    return match.group(0) if match else None

def extract_item_7_to_7a(text: str) -> str:
    pattern = r'(?s)((?:I\s*T\s*E\s*M|ITEM)\s*7\.)(.*?)(?=(?:I\s*T\s*E\s*M|ITEM)\s*7a\.)'
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        return match.group(0)
    return match.group(0) if match else None

def extract_filing_date(original_text: str) -> datetime:
    match = re.search(r'<FILING-DATE>(\d{8})', original_text)
    if match:
        return pd.to_datetime(match.group(1), format='%Y%m%d')
    match = re.search(r'<ACCEPTANCE-DATETIME>(\d{8})', original_text)
    if match:
        return pd.to_datetime(match.group(1), format='%Y%m%d')

    soup = BeautifulSoup(original_text, 'html.parser')
    for tag in soup.find_all(text=re.compile(r'(FILING-DATE|ACCEPTANCE-DATETIME)')):
        match = re.search(r'\d{8}', tag)
        if match:
            return pd.to_datetime(match.group(0), format='%Y%m%d')
    return None

def clean_and_save_filings(input_dir: str, output_dir: str, company_name: str, cik: str) -> List[Dict]:
    filings = []
    input_path = Path(input_dir)
    output_base = Path(output_dir)

    for file_path in input_path.rglob("*.txt"):
        filing_date_str = file_path.parent.name
        try:
            filing_date = datetime.strptime(filing_date_str, "%Y-%m-%d")
        except ValueError:
            print(f"Omitiendo {file_path}: Nombre de carpeta no es una fecha válida")
            continue

        try:
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                original_text = f.read()

            raw_text = extract_10k_text(original_text)
            if not raw_text:
                print(f"Omitiendo {file_path}: No se extrajo texto 10-K válido")
                continue

            text_cleaned = clean_html_with_bs(raw_text)
            relevant_text = extract_relevant_text(text_cleaned)
            item_7_text = extract_item_7_to_7a(relevant_text)

            if not item_7_text:
                print(f"Omitiendo {file_path}: No se extrajo sección ITEM 7 a 7A")
                continue

            output_dir = output_base / filing_date_str
            output_dir.mkdir(parents=True, exist_ok=True)

            filing_year = filing_date.year
            output_filename = f"{company_name.replace(' ', '_')}_{filing_year}_item7.txt"
            output_file_path = output_dir / output_filename

            with open(output_file_path, 'w', encoding='utf-8') as f_out:
                f_out.write(item_7_text)

            filings.append({
                "date": filing_date,
                "year": filing_year,
                "company": company_name,
                "CIK": cik,
                "file_path": str(output_file_path)
            })
            print(f"Procesado {file_path} -> {output_file_path}")

        except Exception as e:
            print(f"Error procesando {file_path}: {e}")

    return filings

def main():
    companies = load_companies_from_excel(COMPANIES_EXCEL)
    if not companies:
        print("No se cargaron empresas. Terminando ejecución.")
        return

    all_cleaned_filings = []
    for company in companies:
        input_dir = str(Path(BASE_DIR) / company["input_dir"].lstrip("/\\"))
        output_dir = str(Path(BASE_DIR) / company["output_dir"].lstrip("/\\"))

        cleaned = clean_and_save_filings(
            input_dir=input_dir,
            output_dir=output_dir,
            company_name=company["company_name"],
            cik=company["cik"]
        )
        all_cleaned_filings.extend(cleaned)

    if all_cleaned_filings:
        df_10k_clean = pd.DataFrame(all_cleaned_filings)
        try:
            df_10k_clean.to_excel(OUTPUT_EXCEL, index=False)
            print(f"Metadatos guardados en {OUTPUT_EXCEL}")
        except Exception as e:
            print(f"Error guardando metadatos en {OUTPUT_EXCEL}: {e}")
        return df_10k_clean
    else:
        print("No se procesaron formularios. No se creó el archivo Excel.")
        return None

if __name__ == "__main__":
    df_10k_clean = main()


Procesado D:\TFM\Analisis_10k\Input\BlackRock\2025-02-25\full-submission.txt -> D:\TFM\Analisis_10k\Input_clean\BlackRock\2025-02-25\BlackRock_2025_item7.txt
Procesado D:\TFM\Analisis_10k\Input\Smurfit_Westrock\2025-03-07\full-submission.txt -> D:\TFM\Analisis_10k\Input_clean\Smurfit_Westrock\2025-03-07\Smurfit_Westrock_2025_item7.txt
Procesado D:\TFM\Analisis_10k\Input\Bunge_Global\2024-02-22\full-submission.txt -> D:\TFM\Analisis_10k\Input_clean\Bunge_Global\2024-02-22\Bunge_Global_2024_item7.txt
Procesado D:\TFM\Analisis_10k\Input\Bunge_Global\2025-02-20\full-submission.txt -> D:\TFM\Analisis_10k\Input_clean\Bunge_Global\2025-02-20\Bunge_Global_2025_item7.txt
Procesado D:\TFM\Analisis_10k\Input\GE_Vernova\2025-02-06\full-submission.txt -> D:\TFM\Analisis_10k\Input_clean\GE_Vernova\2025-02-06\GE_Vernova_2025_item7.txt
Procesado D:\TFM\Analisis_10k\Input\TKO_Group_Holdings\2024-02-27\full-submission.txt -> D:\TFM\Analisis_10k\Input_clean\TKO_Group_Holdings\2024-02-27\TKO_Group_Holding

In [18]:
df_10k_clean.head

,date,year,company,CIK,file_path
0,2025-02-25,2025,BlackRock,2012383,D:\TFM\Analisis_10k\Input_clean\BlackRock\2025...
1,2025-03-07,2025,Smurfit_Westrock,2005951,D:\TFM\Analisis_10k\Input_clean\Smurfit_Westro...
2,2024-02-22,2024,Bunge_Global,1996862,D:\TFM\Analisis_10k\Input_clean\Bunge_Global\2...
3,2025-02-20,2025,Bunge_Global,1996862,D:\TFM\Analisis_10k\Input_clean\Bunge_Global\2...
4,2025-02-06,2025,GE_Vernova,1996810,D:\TFM\Analisis_10k\Input_clean\GE_Vernova\202...
...,...,...,...,...,...
4206,2021-02-19,2021,Abbott_Laboratories,1800,D:\TFM\Analisis_10k\Input_clean\Abbott_Laborat...
4207,2022-02-18,2022,Abbott_Laboratories,1800,D:\TFM\Analisis_10k\Input_clean\Abbott_Laborat...
4208,2023-02-17,2023,Abbott_Laboratories,1800,D:\TFM\Analisis_10k\Input_clean\Abbott_Laborat...
4209,2024-02-16,2024,Abbott_Laboratories,1800,D:\TFM\Analisis_10k\Input_clean\Abbott_Laborat...
